In [4]:
import numpy as np #linear algebra
import pandas as pd #data processing
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import tensorflow as tf

In [5]:
train = pd.read_csv("../input/digit-recognizer/train.csv")
test = pd.read_csv("../input/digit-recognizer/test.csv")
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
print(train.shape)
print(test.shape)

(42000, 785)
(28000, 784)


In [7]:
train_y = train['label'].astype('float32')
train_x = train.drop(['label'],axis=1).astype('int32')
test_x = test.astype('float32')
train_x.shape, train_y.shape, test_x.shape 

((42000, 784), (42000,), (28000, 784))

In [8]:
##reshaping for gray scale image
train_x = train_x.values.reshape(-1,28,28,1)
train_x = train_x/255.0 #normalaising
test_x = test_x.values.reshape(-1,28,28,1)
test_x = test_x/255.0
train_x.shape, test_x.shape

((42000, 28, 28, 1), (28000, 28, 28, 1))

In [9]:
#one hot encoding
train_y = tf.keras.utils.to_categorical(train_y,10)
train_y.shape

(42000, 10)

In [10]:
#printing original labels of top 5 row
print(train['label'].head())

0    1
1    0
2    1
3    4
4    0
Name: label, dtype: int64


In [11]:
#one hot encoding of the same labels
print(train_y[0:5,:])

[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [12]:
#defining the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(28,28,1)),
    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu',padding='Same'),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu',padding='Same'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    #tf.keras.layers.Dropout(0.50),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.50),
    tf.keras.layers.Dense(10,activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
dropout (Dropout)            (None, 6, 6, 64)          0

In [13]:
#defining the callback function to stop our training once the acceptable accuracy is reached
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs={}):
        if(logs.get('accuracy')>0.999):
            print("\nReached 99.9% accuracy so cancelling training:")
            self.model.stop_training = True
callbacks = myCallback()
    

In [14]:
#compiling and model training with batch size=50, epochs=20, optimizer=adam
Optimizer = tf.keras.optimizers.Adam(
learning_rate=0.0005,
beta_1=0.9,
beta_2=0.999,
epsilon=1e-07,
name='Adam'
)
model.compile(optimizer=Optimizer, loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(train_x, train_y, batch_size=50, epochs=20, callbacks=[callbacks])

Epoch 1/20
840/840 [==============================] - 67s 79ms/step - loss: 0.5894 - accuracy: 0.8010
Epoch 2/20
840/840 [==============================] - 65s 78ms/step - loss: 0.0709 - accuracy: 0.9789
Epoch 3/20
840/840 [==============================] - 66s 79ms/step - loss: 0.0493 - accuracy: 0.9847
Epoch 4/20
840/840 [==============================] - 67s 79ms/step - loss: 0.0378 - accuracy: 0.9882
Epoch 5/20
840/840 [==============================] - 67s 79ms/step - loss: 0.0290 - accuracy: 0.9911
Epoch 6/20
840/840 [==============================] - 66s 79ms/step - loss: 0.0241 - accuracy: 0.9923
Epoch 7/20
840/840 [==============================] - 66s 79ms/step - loss: 0.0207 - accuracy: 0.9934
Epoch 8/20
840/840 [==============================] - 66s 79ms/step - loss: 0.0172 - accuracy: 0.9945
Epoch 9/20
840/840 [==============================] - 66s 79ms/step - loss: 0.0146 - accuracy: 0.9955
Epoch 10/20
840/840 [==============================] - 66s 79ms/step - loss: 0.014

In [15]:
results = model.predict(test_x)

#select the index with maximum probability
results = np.argmax(results, axis=1)
results = pd.Series(results,name="Label")

In [16]:
submission = pd.concat([pd.Series(range(1,28001),name="ImageId"),results],axis=1)
submission.to_csv("submission.csv",index=False)